In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/Notebooks/Luodingo
!ls

/content/gdrive/My Drive/Colab Notebooks/Notebooks/Luodingo
 Data	       trainer.py	    transformer-model.pt
 __pycache__  'Train Model.ipynb'   transformer.py


In [0]:
import torch 
import torch.optim as optim

from transformer import Transformer
from trainer import TrainModel

from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

%load_ext autoreload
%autoreload 2

In [0]:
TEXT = Field(sequential=True, tokenize=lambda x: x.split(), lower=True)
LABEL = LabelField(dtype=torch.long)
fields = [('keyword', TEXT), ('label', LABEL)]

In [0]:
train, val, test = TabularDataset.splits(
                            path='Data',
                            train='train.csv',
                            validation='val.csv',
                            test='test.csv',
                            format='csv',
                            skip_header=True,
                            fields=fields)

In [6]:
TEXT.build_vocab(train, 
                 min_freq = 3,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(val)

.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 398687/400000 [00:25<00:00, 17585.84it/s]

In [0]:
BATCH_SIZE = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = BucketIterator.splits(
                            (train, val, test),
                            batch_size=BATCH_SIZE,
                            device=device,
                            sort_key=lambda x: len(x.keyword))

In [10]:
print(train[10].__dict__)
print(train[10].label)
print(TEXT.vocab.vectors.shape)

{'keyword': ['dogs', 'vegan', 'insted', 'they', 'salads', 'meat', 'eat'], 'label': '6'}
6
torch.Size([3224, 100])


In [0]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(LABEL.vocab)
HID_DIM = 1024
N_LAYERS = 2
N_HEADS = 4
DROPOUT_RATE = 0.1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

transformer = Transformer(input_dim = INPUT_DIM, 
                          output_dim = OUTPUT_DIM,
                          hid_dim = HID_DIM, 
                          n_heads = N_HEADS,
                          n_layers = N_LAYERS,
                          dropout_rate = DROPOUT_RATE,
                          device = device,
                          pad_idx = PAD_IDX)

In [0]:
LEARNING_RATE = 0.0001

adam = torch.optim.Adam(transformer.parameters(), lr=LEARNING_RATE)
cross_e = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [0]:
trainer = TrainModel(
            model=transformer,
            train_iterator=train_iter,
            val_iterator=val_iter,
            optimizer=adam,
            criterion=cross_e)

In [14]:
N_EPOCHS = 10
CLIP = 1
trainer.epoch(n_epochs=N_EPOCHS, clip=CLIP, model_name='transformer-model.pt')

Epoch: 01 | Time: 0m 23s
	Train Loss: 8.344 | Train PPL: 4205.549
	 Val. Loss: 8.344 |  Val. PPL: 4203.373
Epoch: 02 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4201.517
	 Val. Loss: 8.343 |  Val. PPL: 4200.428
Epoch: 03 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4200.116
	 Val. Loss: 8.343 |  Val. PPL: 4200.074
Epoch: 04 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.883
	 Val. Loss: 8.343 |  Val. PPL: 4199.982
Epoch: 05 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.821
	 Val. Loss: 8.343 |  Val. PPL: 4199.971
Epoch: 06 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.802
	 Val. Loss: 8.343 |  Val. PPL: 4200.021
Epoch: 07 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.802
	 Val. Loss: 8.343 |  Val. PPL: 4200.049
Epoch: 08 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.803
	 Val. Loss: 8.343 |  Val. PPL: 4200.092
Epoch: 09 | Time: 0m 23s
	Train Loss: 8.343 | Train PPL: 4199.811
	 Val. Loss: 8.343 |  Val. PPL: 4200.113
Epoch: 10 | Time: 0m 23s
	Train Loss: